# Import relevant libaries

In [1]:
# we must import the libraries once again since we haven't imported them in this file
import numpy as np
import tensorflow as tf

## Load .npz files and store each in variable

In [2]:
# let's create a temporary variable npz, where we will store each of the three HR datasets
npz = np.load('HR_data_train.npz')

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(np.float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(np.int)

# we load the validation data in the temporary variable
npz = np.load('HR_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# we load the test data in the temporary variable
npz = np.load('HR_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

## Training the model


### Dropout(0.2) and early_stopping used to prevent overfitting of the model

In [94]:
# Set the input and output sizes
input_size = 16
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 20
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dropout(0.2),
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 32

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Train on 5520 samples, validate on 690 samples
Epoch 1/100
5520/5520 - 1s - loss: 9.7704 - accuracy: 0.4969 - val_loss: 0.7153 - val_accuracy: 0.4246
Epoch 2/100
5520/5520 - 0s - loss: 2.4929 - accuracy: 0.5049 - val_loss: 0.7010 - val_accuracy: 0.5101
Epoch 3/100
5520/5520 - 0s - loss: 1.2742 - accuracy: 0.5130 - val_loss: 0.6861 - val_accuracy: 0.5464
Epoch 4/100
5520/5520 - 0s - loss: 0.9000 - accuracy: 0.5040 - val_loss: 0.6847 - val_accuracy: 0.5493
Epoch 5/100
5520/5520 - 0s - loss: 0.7954 - accuracy: 0.5134 - val_loss: 0.6804 - val_accuracy: 0.6101
Epoch 6/100
5520/5520 - 0s - loss: 0.7468 - accuracy: 0.5317 - val_loss: 0.6824 - val_accuracy: 0.5101
Epoch 7/100
5520/5520 - 0s - loss: 0.7273 - accuracy: 0.5406 - val_loss: 0.6747 - val_accuracy: 0.5913
Epoch 8/100
5520/5520 - 0s - loss: 0.7055 - accuracy: 0.5498 - val_loss: 0.6723 - val_accuracy: 0.7304
Epoch 9/100
5520/5520 - 0s - loss: 0.7060 - accuracy: 0.5524 - val_loss: 0.6683 - val_accuracy: 0.5667
Epoch 10/100
5520/5520 - 0

## Model Testing

In [95]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

690/690 [==============================] - 0s 20us/sample - loss: 0.3964 - accuracy: 0.8667


In [96]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.40. Test accuracy: 86.67%
